## **Federated Learning for attack detection: 5 nodes sharing gradients: Bot-IoT dataset**

IDs from this file = **Bot_pred5_niter_nepoch**, with niter being number of iterations while training, and nepoch number of epochs for local models. 

### Static elements (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') |  (data['proto'] =='igmp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['attack']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','saddr','sport','daddr','dport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    data_features = data_features.fillna(value=-1)
    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['saddr'] = data_features['saddr'].apply(lambda x: x.split(".")[-1])
    data_features['saddr'] = data_features['saddr'].apply(lambda x: x.split(":")[-1])
    data_features['saddr'] = data_features['saddr'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['daddr'] = data_features['daddr'].apply(lambda x: x.split(".")[-1])
    data_features['daddr'] = data_features['daddr'].apply(lambda x: x.split(":")[-1])
    data_features['daddr'] = data_features['daddr'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dport'] = data_features['dport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dport'] = data_features['dport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dport'] = data_features['dport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['saddr'] = data_features['saddr'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['daddr'] = data_features['daddr'].astype(int)
    data_features['dport'] = data_features['dport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['attack']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)


    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [5]:
bot1 = pd.read_csv('../datasets/Bot-IoT/pred5_part1.csv')
bot2 = pd.read_csv('../datasets/Bot-IoT/pred5_part2.csv')
bot3 = pd.read_csv('../datasets/Bot-IoT/pred5_part3.csv')
bot4 = pd.read_csv('../datasets/Bot-IoT/pred5_part4.csv')
bot5 = pd.read_csv('../datasets/Bot-IoT/pred5_part5.csv')
test = pd.read_csv('../datasets/Bot-IoT/Bot_test.csv')

In [6]:
x1, y1 = preprocessing(bot1)
x2, y2 = preprocessing(bot2)
x3, y3 = preprocessing(bot3)
x4, y4 = preprocessing(bot4)
x5, y5 = preprocessing(bot5)

In [7]:
xtest, ytest = preprocessing(test)

In [8]:
def weiszfeld_update(points, tol=1e-5, max_iter=100):
    """
    Compute the geometric median using Weiszfeld's algorithm.

    Args:
    points: A list of points (numpy arrays) to find the geometric median of.
    tol: Tolerance for stopping criterion.
    max_iter: Maximum number of iterations.

    Returns:
    The geometric median of the points.
    """
    points = np.array(points)
    median = np.mean(points, axis=0)

    for _ in range(max_iter):
        distances = np.linalg.norm(points - median, axis=1)
        nonzero_distances = np.where(distances != 0, distances, np.finfo(float).eps)
        weights = 1 / nonzero_distances
        new_median = np.sum(points * weights[:, None], axis=0) / np.sum(weights)

        if np.linalg.norm(new_median - median) < tol:
            return new_median

        median = new_median

    return median

def aggregate(grad_list):
    """
    Apply Geometric Median aggregation method to a list of gradients.

    Args:
    grad_list: List of gradients from different models. Each element in the list
               is a list of gradients for each layer of a model.

    Returns:
    The geometric median of the gradients.
    """
    # Flatten gradients to compute geometric median
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute geometric median using Weiszfeld's algorithm
    flat_median = weiszfeld_update(flat_grads)

    # Reshape the flat median back to the original shape
    median_grad = []
    index = 0
    for grad in grad_list[0]:
        shape = tf.shape(grad)
        size = tf.reduce_prod(shape)
        median_grad.append(tf.reshape(flat_median[index:index + size], shape))
        index += size

    return median_grad

### Experiments

#### niter = 3, nepoch = 5

In [9]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_pred5_3_5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 2 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [13]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_pred5_3_5.hdf5')

Epoch 1/5
3/3 [==============================] - 1s 199ms/step - loss: 0.6760 - accuracy: 0.7526 - val_loss: 0.7515 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============================] - 0s 94ms/step - loss: 0.6042 - accuracy: 0.9153 - val_loss: 0.8475 - val_accuracy: 0.1958
Epoch 3/5
3/3 [==============================] - 0s 95ms/step - loss: 0.5129 - accuracy: 0.9153 - val_loss: 1.0246 - val_accuracy: 0.1958
Epoch 1/5
3/3 [==============================] - 1s 184ms/step - loss: 0.4224 - accuracy: 0.9153 - val_loss: 1.3524 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============================] - 0s 113ms/step - loss: 0.3353 - accuracy: 0.9153 - val_loss: 1.7716 - val_accuracy: 0.1958
Epoch 3/5
3/3 [==============================] - 0s 105ms/step - loss: 0.2931 - accuracy: 0.9153 - val_loss: 2.0909 - val_accuracy: 0.1958
Epoch 1/5
3/3 [==============================] - 1s 192ms/step - loss: 0.2818 - accuracy: 0.9153 - val_loss: 1.5791 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============

In [14]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.04243539273738861], [0.02166404202580452], [0.016052810475230217]]
Accuracy for iterations:  [[0.9850848317146301], [0.9951257705688477], [0.9961006045341492]]
F1 for iterations:  [[0.9849378997652537], [0.9951264571404197], [0.9961022861186998]]
Precision for iterations:  [[0.9852502793249531], [0.9951272636275589], [0.9961049120986155]]
Recall for iterations:  [[0.9850848118541626], [0.9951257555078963], [0.996100604406317]]


#### niter = 5, nepoch = 5

In [15]:
# Define 
num_nodes = 5
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [16]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_pred5_5_5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [17]:
global_model = build_model((24,1,1))

In [18]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [19]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_pred5_5_5.hdf5')

Epoch 1/5
3/3 [==============================] - 2s 590ms/step - loss: 0.6830 - accuracy: 0.5696 - val_loss: 0.7355 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============================] - 1s 308ms/step - loss: 0.6144 - accuracy: 0.9163 - val_loss: 0.8151 - val_accuracy: 0.1958
Epoch 3/5
3/3 [==============================] - 1s 318ms/step - loss: 0.5305 - accuracy: 0.9161 - val_loss: 0.9638 - val_accuracy: 0.1958
Epoch 4/5
3/3 [==============================] - 1s 308ms/step - loss: 0.4301 - accuracy: 0.9155 - val_loss: 1.2338 - val_accuracy: 0.1958
Epoch 5/5
3/3 [==============================] - 1s 273ms/step - loss: 0.3386 - accuracy: 0.9153 - val_loss: 1.6533 - val_accuracy: 0.1958
Epoch 1/5
3/3 [==============================] - 1s 483ms/step - loss: 0.3009 - accuracy: 0.9153 - val_loss: 2.0083 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============================] - 1s 227ms/step - loss: 0.2787 - accuracy: 0.9153 - val_loss: 1.8345 - val_accuracy: 0.1958
Epoch 3/5
3/3 [============

In [20]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.03968409076333046], [0.021323535591363907], [0.014177300035953522], [0.011007408611476421], [0.01291663758456707]]
Accuracy for iterations:  [[0.9863520860671997], [0.9950282573699951], [0.9964905381202698], [0.9973679184913635], [0.9977578520774841]]
F1 for iterations:  [[0.9862272925716498], [0.9950279124334843], [0.9964930632306184], [0.9973711126806949], [0.9977599378678729]]
Precision for iterations:  [[0.9865006814266735], [0.9950275804804384], [0.996498204305952], [0.997381891235], [0.9977664653229681]]
Recall for iterations:  [[0.9863521154221095], [0.9950282706180542], [0.9964905439656854], [0.997367907974264], [0.9977578475336323]]


#### niter = 5, nepoch = 10

In [21]:
# Define 
num_nodes = 5
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [22]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_pred5_5_10.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [23]:
global_model = build_model((24,1,1))

In [24]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [25]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_pred5_5_10.hdf5')

Epoch 1/10
3/3 [==============================] - 1s 361ms/step - loss: 0.6832 - accuracy: 0.5614 - val_loss: 0.7252 - val_accuracy: 0.1958
Epoch 2/10
3/3 [==============================] - 1s 281ms/step - loss: 0.6203 - accuracy: 0.9163 - val_loss: 0.7818 - val_accuracy: 0.1958
Epoch 3/10
3/3 [==============================] - 1s 199ms/step - loss: 0.5439 - accuracy: 0.9163 - val_loss: 0.8892 - val_accuracy: 0.1958
Epoch 4/10
3/3 [==============================] - 0s 153ms/step - loss: 0.4492 - accuracy: 0.9157 - val_loss: 1.0816 - val_accuracy: 0.1958
Epoch 5/10
3/3 [==============================] - 0s 163ms/step - loss: 0.3537 - accuracy: 0.9153 - val_loss: 1.3815 - val_accuracy: 0.1958
Epoch 6/10
3/3 [==============================] - 1s 202ms/step - loss: 0.2871 - accuracy: 0.9153 - val_loss: 1.7284 - val_accuracy: 0.1958
Epoch 7/10
3/3 [==============================] - 1s 284ms/step - loss: 0.2592 - accuracy: 0.9153 - val_loss: 1.9111 - val_accuracy: 0.1958
Epoch 8/10
3/3 [====

In [26]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.02185605652630329], [0.012391968630254269], [0.009844987653195858], [0.027016656473279], [0.02136952430009842]]
Accuracy for iterations:  [[0.994735836982727], [0.9971729516983032], [0.9980502724647522], [0.9728017449378967], [0.9990251660346985]]
F1 for iterations:  [[0.9947335366906417], [0.9971755738334052], [0.9980522590657185], [0.9732909075431306], [0.9990255715296751]]
Precision for iterations:  [[0.9947322026220543], [0.9971826419600667], [0.9980593631596173], [0.9755319603590986], [0.9990269411789433]]
Recall for iterations:  [[0.994735815948528], [0.9971729381945799], [0.9980503022031585], [0.9728017157340613], [0.9990251511015793]]


#### niter = 5, nepoch = 15

In [27]:
# Define 
num_nodes = 5
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [28]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_pred5_5_15.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [29]:
global_model = build_model((24,1,1))

In [30]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [31]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_pred5_5_15.hdf5')

Epoch 1/15
3/3 [==============================] - 1s 496ms/step - loss: 0.6823 - accuracy: 0.6017 - val_loss: 0.7223 - val_accuracy: 0.1958
Epoch 2/15
3/3 [==============================] - 1s 210ms/step - loss: 0.6223 - accuracy: 0.9163 - val_loss: 0.7874 - val_accuracy: 0.1958
Epoch 3/15
3/3 [==============================] - 1s 183ms/step - loss: 0.5419 - accuracy: 0.9163 - val_loss: 0.9139 - val_accuracy: 0.1958
Epoch 4/15
3/3 [==============================] - 1s 270ms/step - loss: 0.4414 - accuracy: 0.9157 - val_loss: 1.1411 - val_accuracy: 0.1958
Epoch 5/15
3/3 [==============================] - 1s 193ms/step - loss: 0.3457 - accuracy: 0.9153 - val_loss: 1.4894 - val_accuracy: 0.1958
Epoch 6/15
3/3 [==============================] - 1s 242ms/step - loss: 0.2847 - accuracy: 0.9153 - val_loss: 1.8563 - val_accuracy: 0.1958
Epoch 7/15
3/3 [==============================] - 1s 204ms/step - loss: 0.2648 - accuracy: 0.9153 - val_loss: 1.9842 - val_accuracy: 0.1958
Epoch 8/15
3/3 [====

In [32]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.01900262013077736], [0.010455472394824028], [0.02024257183074951], [0.04036456719040871], [0.06192406266927719]]
Accuracy for iterations:  [[0.9960030913352966], [0.9973679184913635], [0.998342752456665], [0.9741665124893188], [0.9742639660835266]]
F1 for iterations:  [[0.9960051299054018], [0.9973714873597711], [0.9983440650622382], [0.9746155867515299], [0.9747098208615351]]
Precision for iterations:  [[0.9960084249564818], [0.9973845221412558], [0.9983485552559466], [0.9766981706368265], [0.9767774065443906]]
Recall for iterations:  [[0.9960031195164749], [0.997367907974264], [0.9983427568726847], [0.9741665041918502], [0.9742639890816923]]


#### niter = 10, nepoch = 5

In [33]:
# Define 
num_nodes = 5
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [34]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_pred5_10_5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [35]:
global_model = build_model((24,1,1))

In [36]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [37]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_pred5_10_5.hdf5')

Epoch 1/5


3/3 [==============================] - 1s 335ms/step - loss: 0.6726 - accuracy: 0.9133 - val_loss: 0.7433 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============================] - 1s 200ms/step - loss: 0.6033 - accuracy: 0.9153 - val_loss: 0.8296 - val_accuracy: 0.1958
Epoch 3/5
3/3 [==============================] - 1s 196ms/step - loss: 0.5125 - accuracy: 0.9153 - val_loss: 0.9902 - val_accuracy: 0.1958
Epoch 4/5
3/3 [==============================] - 0s 147ms/step - loss: 0.4072 - accuracy: 0.9153 - val_loss: 1.2674 - val_accuracy: 0.1958
Epoch 5/5
3/3 [==============================] - 0s 148ms/step - loss: 0.3187 - accuracy: 0.9153 - val_loss: 1.6603 - val_accuracy: 0.1958
Epoch 1/5
3/3 [==============================] - 1s 493ms/step - loss: 0.2904 - accuracy: 0.9153 - val_loss: 1.8862 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============================] - 1s 194ms/step - loss: 0.2627 - accuracy: 0.9153 - val_loss: 1.6457 - val_accuracy: 0.1958
Epoch 3/5
3/3 [======================

In [38]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.032635290175676346], [0.02051631174981594], [0.01650966703891754], [0.011549423448741436], [0.00872776098549366], [0.008234844543039799], [0.00799685250967741], [0.009203227236866951], [0.008257870562374592], [0.006040662992745638]]
Accuracy for iterations:  [[0.9861571192741394], [0.9954181909561157], [0.9959056377410889], [0.9969779849052429], [0.9978553056716919], [0.9980502724647522], [0.9980502724647522], [0.9981477856636047], [0.9981477856636047], [0.9985377192497253]]
F1 for iterations:  [[0.9860305396083879], [0.9954198573937628], [0.9959074004246348], [0.9969816478926498], [0.9978568719742669], [0.9980517017947881], [0.9980517017947881], [0.99814977824725], [0.99814977824725], [0.998538880937269]]
Precision for iterations:  [[0.986301998443455], [0.9954221597353692], [0.9959101101599269], [0.9969928959480507], [0.9978610397427016], [0.99805573051938], [0.99805573051938], [0.998157677559427], [0.998157677559427], [0.9985432183332518]]
Recall for iterat

#### niter = 15, nepoch = 5

In [39]:
# Define 
num_nodes = 5
global_updates = 15

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [40]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_pred5_15_5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [41]:
global_model = build_model((24,1,1))

In [42]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [43]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_pred5_15_5.hdf5')

Epoch 1/5
3/3 [==============================] - 1s 310ms/step - loss: 0.6794 - accuracy: 0.7865 - val_loss: 0.7130 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============================] - 0s 151ms/step - loss: 0.6275 - accuracy: 0.9165 - val_loss: 0.7536 - val_accuracy: 0.1958
Epoch 3/5
3/3 [==============================] - 0s 147ms/step - loss: 0.5590 - accuracy: 0.9163 - val_loss: 0.8348 - val_accuracy: 0.1958
Epoch 4/5
3/3 [==============================] - 0s 142ms/step - loss: 0.4695 - accuracy: 0.9163 - val_loss: 0.9908 - val_accuracy: 0.1958
Epoch 5/5
3/3 [==============================] - 0s 135ms/step - loss: 0.3704 - accuracy: 0.9153 - val_loss: 1.2568 - val_accuracy: 0.1958
Epoch 1/5
3/3 [==============================] - 1s 339ms/step - loss: 0.3165 - accuracy: 0.9153 - val_loss: 1.6904 - val_accuracy: 0.1958
Epoch 2/5
3/3 [==============================] - 1s 184ms/step - loss: 0.2726 - accuracy: 0.9153 - val_loss: 1.8701 - val_accuracy: 0.1958
Epoch 3/5
3/3 [============

In [44]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.037362489849328995], [0.022660942748188972], [0.016387399286031723], [0.012209315784275532], [0.009371529333293438], [0.008416160941123962], [0.007575306575745344], [0.0071905311197042465], [0.006221211049705744], [0.0060332827270030975], [0.012586583383381367], [0.014409012161195278], [0.033285871148109436], [0.05641020089387894], [0.05330319702625275]]
Accuracy for iterations:  [[0.9858646988868713], [0.9948332905769348], [0.996393084526062], [0.9967830181121826], [0.9976603388786316], [0.9981477856636047], [0.9986352324485779], [0.9986352324485779], [0.9987326860427856], [0.9987326860427856], [0.998342752456665], [0.9988301992416382], [0.9737765789031982], [0.9737765789031982], [0.9737765789031982]]
F1 for iterations:  [[0.9857343075168505], [0.9948329286073463], [0.9963959063119302], [0.9967869154986272], [0.9976623767357379], [0.9981489841359346], [0.9986356041990007], [0.9986356041990007], [0.9987333338724446], [0.9987333338724446], [0.9983443019023408],

#### niter = 10, nepoch = 10

In [45]:
# Define 
num_nodes = 5
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [46]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_pred5_10_10.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [47]:
global_model = build_model((24,1,1))

In [48]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [49]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_pred5_10_10.hdf5')

Epoch 1/10
3/3 [==============================] - 1s 257ms/step - loss: 0.6765 - accuracy: 0.9106 - val_loss: 0.7217 - val_accuracy: 0.1966
Epoch 2/10
3/3 [==============================] - 0s 154ms/step - loss: 0.6160 - accuracy: 0.9165 - val_loss: 0.7787 - val_accuracy: 0.1958
Epoch 3/10
3/3 [==============================] - 1s 195ms/step - loss: 0.5372 - accuracy: 0.9163 - val_loss: 0.8894 - val_accuracy: 0.1958
Epoch 4/10
3/3 [==============================] - 1s 195ms/step - loss: 0.4396 - accuracy: 0.9153 - val_loss: 1.0883 - val_accuracy: 0.1958
Epoch 5/10
3/3 [==============================] - 1s 189ms/step - loss: 0.3431 - accuracy: 0.9153 - val_loss: 1.3991 - val_accuracy: 0.1958
Epoch 6/10
3/3 [==============================] - 1s 192ms/step - loss: 0.2803 - accuracy: 0.9153 - val_loss: 1.7483 - val_accuracy: 0.1958
Epoch 7/10
3/3 [==============================] - 1s 205ms/step - loss: 0.2550 - accuracy: 0.9153 - val_loss: 1.8921 - val_accuracy: 0.1958
Epoch 8/10
3/3 [====

In [50]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.023114198818802834], [0.01230892539024353], [0.013561930507421494], [0.023924116045236588], [0.03802800178527832], [0.08242013305425644], [0.09051923453807831], [0.10174715518951416], [0.13752664625644684], [0.09053241461515427]]
Accuracy for iterations:  [[0.9941508769989014], [0.9969779849052429], [0.9980502724647522], [0.9844024181365967], [0.9739715456962585], [0.9735816121101379], [0.9737765789031982], [0.973874032497406], [0.9737765789031982], [0.9740689992904663]]
F1 for iterations:  [[0.9941458316186811], [0.9969812171402501], [0.9980519806131148], [0.9845669220416136], [0.9744240062741829], [0.9740472240381023], [0.9742324257968361], [0.9743266363291344], [0.9742292535181346], [0.9745119462689669]]
Precision for iterations:  [[0.9941445399044985], [0.9969903617006493], [0.9980574428164526], [0.9853169058329891], [0.9765094244760937], [0.9761931516149136], [0.9763206783153606], [0.9763998749502693], [0.9762903454510006], [0.9765284108073905]]
Recall fo

#### niter = 15, nepoch = 15

In [51]:
# Define 
num_nodes = 5
global_updates = 15

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [52]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_pred5_15_15.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [53]:
global_model = build_model((24,1,1))

In [54]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [55]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_pred5_15_15.hdf5')

Epoch 1/15
3/3 [==============================] - 1s 397ms/step - loss: 0.6805 - accuracy: 0.7922 - val_loss: 0.7192 - val_accuracy: 0.1958
Epoch 2/15
3/3 [==============================] - 1s 179ms/step - loss: 0.6246 - accuracy: 0.9163 - val_loss: 0.7743 - val_accuracy: 0.1958
Epoch 3/15
3/3 [==============================] - 1s 167ms/step - loss: 0.5500 - accuracy: 0.9161 - val_loss: 0.8791 - val_accuracy: 0.1958
Epoch 4/15
3/3 [==============================] - 1s 177ms/step - loss: 0.4549 - accuracy: 0.9153 - val_loss: 1.0682 - val_accuracy: 0.1958
Epoch 5/15
3/3 [==============================] - 0s 147ms/step - loss: 0.3582 - accuracy: 0.9153 - val_loss: 1.3660 - val_accuracy: 0.1958
Epoch 6/15
3/3 [==============================] - 0s 143ms/step - loss: 0.2880 - accuracy: 0.9153 - val_loss: 1.7309 - val_accuracy: 0.1958
Epoch 7/15
3/3 [==============================] - 0s 144ms/step - loss: 0.2609 - accuracy: 0.9153 - val_loss: 1.9659 - val_accuracy: 0.1958
Epoch 8/15
3/3 [====

In [56]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.01785382255911827], [0.011022340506315231], [0.038738854229450226], [0.045071620494127274], [0.07614639401435852], [0.08003633469343185], [0.08595812320709229], [0.0817970484495163], [0.05749541521072388], [0.06044314429163933], [0.056406475603580475], [0.06548548489809036], [0.12478258460760117], [0.15872572362422943], [0.14053671061992645]]
Accuracy for iterations:  [[0.9957106709480286], [0.9977578520774841], [0.9732891321182251], [0.9741665124893188], [0.9739715456962585], [0.973874032497406], [0.973874032497406], [0.973874032497406], [0.973874032497406], [0.9737765789031982], [0.973874032497406], [0.9739715456962585], [0.9741665124893188], [0.9742639660835266], [0.9742639660835266]]
F1 for iterations:  [[0.9957118989111451], [0.9977602578782502], [0.9737679609816305], [0.9746155867515299], [0.9744271506929901], [0.9743266363291344], [0.9743266363291344], [0.9743234736965571], [0.9743234736965571], [0.9742260769288479], [0.9743203067672298], [0.97442085758